# Analisi sulla somministrazione dei vaccini

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Parametri

In [2]:
data_file='https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-summary-latest.csv'

## Lettura dati e preprocessing

In [4]:
vaccinazioni_data=pd.read_csv(data_file)
print(vaccinazioni_data.columns)

# riordino in base alla data (che creo e faccio indice)
vaccinazioni_data['data'] = pd.to_datetime(vaccinazioni_data['data_somministrazione'])
vaccinazioni_data.index=vaccinazioni_data['data']
vaccinazioni_data=vaccinazioni_data.sort_index()


# creo una colonna col numero di giorni dalla prima consegna
zero_day=vaccinazioni_data['data'][0]
day_counter=[(dd-zero_day).days for dd in vaccinazioni_data['data']]
vaccinazioni_data['day_counter']=day_counter

giorni=pd.unique(vaccinazioni_data['data_somministrazione'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(vaccinazioni_data['area'])


#data[data['denominazione_regione']=='Lombardia'].head(5)
vaccinazioni_data[vaccinazioni_data['area']=='TOS'].tail(5)


Index(['data_somministrazione', 'area', 'totale', 'sesso_maschile',
       'sesso_femminile', 'categoria_operatori_sanitari_sociosanitari',
       'categoria_personale_non_sanitario', 'categoria_ospiti_rsa',
       'categoria_over80', 'categoria_forze_armate',
       'categoria_personale_scolastico', 'prima_dose', 'seconda_dose',
       'codice_NUTS1', 'codice_NUTS2', 'codice_regione_ISTAT', 'nome_area'],
      dtype='object')
Giorni osservati: 73
Da: 2020-12-27
 A: 2021-03-09


,data_somministrazione,area,totale,sesso_maschile,sesso_femminile,categoria_operatori_sanitari_sociosanitari,categoria_personale_non_sanitario,categoria_ospiti_rsa,categoria_over80,categoria_forze_armate,categoria_personale_scolastico,prima_dose,seconda_dose,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,nome_area,data,day_counter
data,,,,,,,,,,,,,,,,,,,
2021-03-05,2021-03-05,TOS,15912,6093,9819,3920,2390,122,4926,682,3872,11184,4728,ITI,ITI1,9,Toscana,2021-03-05,68
2021-03-06,2021-03-06,TOS,7056,2812,4244,3804,1613,116,30,347,1146,2866,4190,ITI,ITI1,9,Toscana,2021-03-06,69
2021-03-07,2021-03-07,TOS,5561,2330,3231,3083,1095,70,3,536,774,2044,3517,ITI,ITI1,9,Toscana,2021-03-07,70
2021-03-08,2021-03-08,TOS,6128,2480,3648,4034,1728,125,54,179,8,1345,4783,ITI,ITI1,9,Toscana,2021-03-08,71
2021-03-09,2021-03-09,TOS,2553,990,1563,1322,581,71,521,55,3,1009,1544,ITI,ITI1,9,Toscana,2021-03-09,72


### Creazione della struttura dati


**MODIFICARE**

- Si crea un dizionario per ciascuno degli indicatori selezionati
- a sua volta ciascun elemento del primo dizionario è un dizionario con le regioni come chiave
    - ogni elemento indicatore contiene i valori dell'indicatore per la regione

**Esempio:**
```    
trend[indicatore][regione]
```

In [6]:
# indicatori osservati
indicatori=['data','day_counter','totale', 'sesso_maschile',
       'sesso_femminile', 'categoria_operatori_sanitari_sociosanitari',
       'categoria_personale_non_sanitario', 'categoria_ospiti_rsa',
       'categoria_over80', 'categoria_forze_armate',
       'categoria_personale_scolastico', 'prima_dose', 'seconda_dose']

# qui immetto tutti i trend per ciascuna regione
vaccinazioni={}
for regione in regioni:
    vaccinazioni[regione]={}
    for ind in indicatori:    
        vaccinazioni[regione][ind]=np.array(vaccinazioni_data[vaccinazioni_data['area']==regione][ind])
      

Aggrego nell'indicatore **ITALIA**

In [18]:
a=np.where(vaccinazioni['TOS']['day_counter']==1)
a[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

## Grafici